##  Joining and Concatenating

En este capítulo exploraremos cómo combinar y unir DataFrames en Polars, una habilidad esencial para trabajar con datos provenientes de múltiples fuentes. Aprenderás:

- Cómo usar `df.join()` para unir DataFrames según valores comunes y diferentes estrategias de combinación.
- Qué es un join asof (`df.join_asof()`), útil para unir DataFrames según el valor más cercano en otra tabla.
- Métodos para concatenar DataFrames: `pl.concat()`, `df.vstack()`, `df.hstack()` y `df.extend()`.
- Cómo unir Series con `series.append()`.
- Las diferencias entre estos métodos y cuándo utilizar cada uno.

Recuerda que los archivos necesarios deben estar en el subdirectorio `data/`.

## Uniones en Polars: `df.join()`

Polars permite combinar DataFrames usando el método `df.join()`, que acepta los siguientes argumentos principales:

| Argumento      | Descripción                                                                 |
|----------------|-----------------------------------------------------------------------------|
| `other`        | DataFrame con el que se va a unir.                                          |
| `on`           | Columna sobre la que se realiza la unión (si el nombre es igual en ambos).  |
| `left_on`/`right_on` | Columnas para unir si los nombres son diferentes en cada DataFrame.    |
| `how`          | Estrategia de unión a utilizar.                                             |
| `suffix`       | Sufijo que se añade a las columnas duplicadas.                              |
| `validate`     | Valida el tipo de unión.                                                    |
| `join_nulls`   | Permite unir valores nulos (por defecto, no se unen).                       |

### Estrategias de unión (`how`)

- **inner**: Solo conserva filas que tienen coincidencia en ambos DataFrames.
- **left**: Conserva todas las filas del DataFrame izquierdo y solo las coincidentes del derecho.
- **right**: Conserva todas las filas del DataFrame derecho y solo las coincidentes del izquierdo.
- **full**: Conserva todas las filas de ambos DataFrames.
- **cross**: Realiza el producto cartesiano (todas las combinaciones posibles).
- **semi**: Conserva filas del DataFrame izquierdo que tienen coincidencia en el derecho.
- **anti**: Conserva filas del DataFrame izquierdo que no tienen coincidencia en el derecho.

> **Nota:** Elige la estrategia de unión según la relación entre tus conjuntos de datos y el resultado que necesitas.

In [27]:
import polars as pl

In [28]:
df_left = pl.DataFrame({"key": ["A", "B", "C", "D"], "value": [1, 2, 3, 4]})
df_right = pl.DataFrame({"key": ["B", "C", "D", "E"], "value": [5, 6, 7, 8]})

print(df_left)
print(df_right)

shape: (4, 2)
┌─────┬───────┐
│ key ┆ value │
│ --- ┆ ---   │
│ str ┆ i64   │
╞═════╪═══════╡
│ A   ┆ 1     │
│ B   ┆ 2     │
│ C   ┆ 3     │
│ D   ┆ 4     │
└─────┴───────┘
shape: (4, 2)
┌─────┬───────┐
│ key ┆ value │
│ --- ┆ ---   │
│ str ┆ i64   │
╞═════╪═══════╡
│ B   ┆ 5     │
│ C   ┆ 6     │
│ D   ┆ 7     │
│ E   ┆ 8     │
└─────┴───────┘


Inner

In [29]:
df_left.join(df_right, on="key", how="inner")

key,value,value_right
str,i64,i64
"""B""",2,5
"""C""",3,6
"""D""",4,7


full

In [30]:
df_left.join(df_right, on="key", how="full")

key,value,key_right,value_right
str,i64,str,i64
"""B""",2,"""B""",5
"""C""",3,"""C""",6
"""D""",4,"""D""",7
null,null,"""E""",8
"""A""",1,null,null


left

In [31]:
df_left.join(df_right, on="key", how="left")

key,value,value_right
str,i64,i64
"""A""",1,null
"""B""",2,5
"""C""",3,6
"""D""",4,7


Right

In [32]:
df_left.join(df_right, on="key", how="right")

value,key,value_right
i64,str,i64
2,"""B""",5
3,"""C""",6
4,"""D""",7
null,"""E""",8


Cross

La estrategia **cross join** en Polars genera el producto cartesiano entre dos DataFrames, es decir, combina cada fila del DataFrame izquierdo con cada fila del derecho. Esto puede resultar en DataFrames muy grandes si los originales tienen muchas filas. No requiere el argumento `on`, ya que todas las combinaciones posibles se generan automáticamente. Es útil para crear todas las permutaciones entre dos conjuntos de datos, por ejemplo, para simulaciones o combinaciones de parámetros. Usa esta funcionalidad con precaución para evitar problemas de memoria.

In [33]:
df_left.join(df_right, how="cross")

key,value,key_right,value_right
str,i64,str,i64
"""A""",1,"""B""",5
"""A""",1,"""C""",6
"""A""",1,"""D""",7
"""A""",1,"""E""",8
"""B""",2,"""B""",5
…,…,…,…
"""C""",3,"""E""",8
"""D""",4,"""B""",5
"""D""",4,"""C""",6


Semi Join

La estrategia **semi join** en Polars permite filtrar el DataFrame izquierdo, conservando únicamente las filas que tienen coincidencia en el DataFrame derecho según la columna clave. A diferencia de otros tipos de join, no añade columnas del DataFrame derecho, solo selecciona las filas del izquierdo que cumplen la condición.

Esto es útil cuando solo necesitas saber qué elementos del DataFrame izquierdo existen en el derecho, sin importar los valores asociados en el derecho.


In [34]:
df_left.join(df_right, on="key", how="semi")

key,value
str,i64
"""B""",2
"""C""",3
"""D""",4


Anti

La estrategia **anti join** en Polars selecciona las filas del DataFrame izquierdo que **no** tienen coincidencia en el DataFrame derecho según la columna clave. Es útil para identificar elementos exclusivos del DataFrame izquierdo, descartando cualquier fila que tenga correspondencia en el derecho. A diferencia del semi join, aquí se filtran las coincidencias y se conservan solo los valores únicos del izquierdo.

In [35]:
df_left.join(df_right, on="key", how="anti")

key,value
str,i64
"""A""",1


## Joinin on Multiple Columns

Puedes unir DataFrames en Polars usando varias columnas como clave, pasando una lista de nombres de columnas al argumento `on`. Por ejemplo:

```python
# Supongamos que ambos DataFrames tienen las columnas "key" y "value"
df_left.join(df_right, on=["key", "value"], how="inner")
```

Esto realizará la unión considerando coincidencias en ambas columnas. Es útil cuando necesitas asegurar que la combinación se haga solo si todos los valores de las columnas especificadas coinciden.

In [36]:
# Ejemplo reproducible de join en múltiples columnas con Polars

residences_left = pl.DataFrame(
    {
        "name": ["Alice", "Bob", "Charlie", "Dave"],
        "city": ["NY", "LA", "NY", "SF"],
        "age": [25, 30, 35, 40],
    }
)

departments_right = pl.DataFrame(
    {
        "name": ["Alice", "Bob", "Charlie", "Dave"],
        "city": ["NY", "LA", "NY", "Chicago"],
        "department": ["Finance", "Marketing", "Engineering", "Operations"],
    }
)

# Join en múltiples columnas: 'name' y 'city'
result = residences_left.join(departments_right, on=["name", "city"], how="inner")
print(residences_left)
print(departments_right)
print(result)

shape: (4, 3)
┌─────────┬──────┬─────┐
│ name    ┆ city ┆ age │
│ ---     ┆ ---  ┆ --- │
│ str     ┆ str  ┆ i64 │
╞═════════╪══════╪═════╡
│ Alice   ┆ NY   ┆ 25  │
│ Bob     ┆ LA   ┆ 30  │
│ Charlie ┆ NY   ┆ 35  │
│ Dave    ┆ SF   ┆ 40  │
└─────────┴──────┴─────┘
shape: (4, 3)
┌─────────┬─────────┬─────────────┐
│ name    ┆ city    ┆ department  │
│ ---     ┆ ---     ┆ ---         │
│ str     ┆ str     ┆ str         │
╞═════════╪═════════╪═════════════╡
│ Alice   ┆ NY      ┆ Finance     │
│ Bob     ┆ LA      ┆ Marketing   │
│ Charlie ┆ NY      ┆ Engineering │
│ Dave    ┆ Chicago ┆ Operations  │
└─────────┴─────────┴─────────────┘
shape: (3, 4)
┌─────────┬──────┬─────┬─────────────┐
│ name    ┆ city ┆ age ┆ department  │
│ ---     ┆ ---  ┆ --- ┆ ---         │
│ str     ┆ str  ┆ i64 ┆ str         │
╞═════════╪══════╪═════╪═════════════╡
│ Alice   ┆ NY   ┆ 25  ┆ Finance     │
│ Bob     ┆ LA   ┆ 30  ┆ Marketing   │
│ Charlie ┆ NY   ┆ 35  ┆ Engineering │
└─────────┴──────┴─────┴────────────

## Validación de Cardinalidad en Joins

Después de unir DataFrames, es importante validar la cardinalidad de la relación entre las tablas para asegurar que la unión cumple con lo esperado. Polars permite validar estas relaciones usando el argumento `validate` en el método `join`.

### Tipos de relaciones y validación

- **Many-to-many (m:m):**  
    Varias filas del DataFrame izquierdo coinciden con varias filas del derecho.  
    *Ejemplo:* Empleados y proyectos (cada empleado puede estar en varios proyectos y cada proyecto puede tener varios empleados).  
    *Nota:* Es la opción por defecto en Polars y no realiza validaciones.

- **One-to-many (1:m):**  
    Una fila del DataFrame izquierdo coincide con varias filas del derecho.  
    *Ejemplo:* Departamentos y empleados (cada departamento tiene varios empleados, pero cada empleado pertenece a un solo departamento).  
    *Validación:* Polars verifica que los valores de unión sean únicos en el DataFrame izquierdo.

- **Many-to-one (m:1):**  
    Varias filas del DataFrame izquierdo coinciden con una fila del derecho.  
    *Ejemplo:* Empleados y ciudades (cada empleado vive en una ciudad, pero una ciudad puede tener varios empleados).  
    *Validación:* Polars verifica que los valores de unión sean únicos en el DataFrame derecho.

- **One-to-one (1:1):**  
    Una fila del DataFrame izquierdo coincide con una fila del derecho.  
    *Ejemplo:* Empleados y IDs de empleados (cada empleado tiene un solo ID y viceversa).  
    *Validación:* Polars verifica que los valores de unión sean únicos en ambos DataFrames.

### Uso del argumento `validate`

Para validar la cardinalidad, pasa el argumento `validate` al método `join` con el tipo de relación esperado:

```python
df_left.join(df_right, on="columna_clave", how="inner", validate="1:m")
```

Esto ayuda a detectar errores de modelado y asegurar la integridad de los datos tras la unión.

In [37]:
employees = pl.DataFrame(
    {
        "employee_id": [1, 2, 3, 4],
        "name": ["Alice", "Bob", "Charlie", "Dave"],
        "department_id": [10, 10, 30, 10],
    }
)

departments = pl.DataFrame(
    {
        "department_id": [10, 20, 30],
        "department_name": [
            "Information Technology",
            "Finance",
            "Human Resources",
        ],
    }
)

# Left join with cardinality validation (many-to-one)
print(employees)
print(departments)
employees.join(departments, on="department_id", how="left", validate="m:1")

shape: (4, 3)
┌─────────────┬─────────┬───────────────┐
│ employee_id ┆ name    ┆ department_id │
│ ---         ┆ ---     ┆ ---           │
│ i64         ┆ str     ┆ i64           │
╞═════════════╪═════════╪═══════════════╡
│ 1           ┆ Alice   ┆ 10            │
│ 2           ┆ Bob     ┆ 10            │
│ 3           ┆ Charlie ┆ 30            │
│ 4           ┆ Dave    ┆ 10            │
└─────────────┴─────────┴───────────────┘
shape: (3, 2)
┌───────────────┬────────────────────────┐
│ department_id ┆ department_name        │
│ ---           ┆ ---                    │
│ i64           ┆ str                    │
╞═══════════════╪════════════════════════╡
│ 10            ┆ Information Technology │
│ 20            ┆ Finance                │
│ 30            ┆ Human Resources        │
└───────────────┴────────────────────────┘


employee_id,name,department_id,department_name
i64,str,i64,str
1,"""Alice""",10,"""Information Technology"""
2,"""Bob""",10,"""Information Technology"""
3,"""Charlie""",30,"""Human Resources"""
4,"""Dave""",10,"""Information Technology"""


In [38]:

try:
    departments = pl.DataFrame(
        {
            "department_id": [10, 20, 10],
            "department_name": [
                "Information Technology",
                "Finance",
                "Human Resources",
            ],
        }
    )
    # departments ya está definido en el notebook, no es necesario redefinirlo
    result = employees.join(departments, on="department_id", how="left", validate="m:1")
    print(result)
except Exception as error:
    print("Error al unir los DataFrames:", error)

Error al unir los DataFrames: join keys did not fulfill m:1 validation


## Uniones Inexactas con `df.join_asof()`

En ocasiones, los valores clave para unir dos DataFrames no coinciden exactamente, como sucede con marcas de tiempo registradas en diferentes momentos por sistemas distintos. Para estos casos, Polars ofrece el método `df.join_asof()`, que permite unir filas basándose en el valor más cercano de una columna clave.

### Argumentos principales de `df.join_asof()`

| Argumento         | Descripción                                                                                  |
|-------------------|----------------------------------------------------------------------------------------------|
| `other`           | DataFrame con el que se va a unir.                                                           |
| `on`              | Columna sobre la que se realiza la unión (si el nombre es igual en ambos).                   |
| `left_on`/`right_on` | Columnas para unir si los nombres son diferentes en cada DataFrame.                        |
| `by`              | Columnas para agrupar antes de realizar el join asof.                                        |
| `by_left`/`by_right` | Columnas para agrupar antes del join asof si los nombres difieren.                        |
| `strategy`        | Estrategia de unión (`forward`, `backward`, `nearest`).                                      |
| `suffix`          | Sufijo para columnas duplicadas.                                                             |
| `tolerance`       | Diferencia máxima entre valores para considerarlos coincidencia.                             |
| `allow_parallel`  | Permite cálculo paralelo hasta el join (por defecto `True`).                                 |
| `force_parallel`  | Fuerza el cálculo paralelo antes del join (por defecto `False`).                             |

> **Importante:** Ambos DataFrames deben estar ordenados por la columna clave antes de usar `df.join_asof()`.

Esta técnica es especialmente útil para unir series temporales o datos donde la coincidencia exacta no es posible, pero se requiere la mejor aproximación.

In [39]:
# Sobrescribimos df_left y df_right con nuevos valores
df_left = pl.DataFrame({"int_id": [10, 5], "value": ["b", "a"]})
df_right = pl.DataFrame({"int_id": [4, 7, 12], "value": [1, 2, 3]})
print(df_left)
print(df_right)


shape: (2, 2)
┌────────┬───────┐
│ int_id ┆ value │
│ ---    ┆ ---   │
│ i64    ┆ str   │
╞════════╪═══════╡
│ 10     ┆ b     │
│ 5      ┆ a     │
└────────┴───────┘
shape: (3, 2)
┌────────┬───────┐
│ int_id ┆ value │
│ ---    ┆ ---   │
│ i64    ┆ i64   │
╞════════╪═══════╡
│ 4      ┆ 1     │
│ 7      ┆ 2     │
│ 12     ┆ 3     │
└────────┴───────┘


In [40]:
try:
    result = df_left.join_asof(df_right, on="int_id", tolerance=3)
    print(result)
except Exception as error:
    print("Error al realizar join_asof:", error)

Error al realizar join_asof: argument in operation 'asof_join' is not sorted, please sort the 'expr/series/column' first


In [41]:
df_left = df_left.sort("int_id")
df_right = df_right
df_left.join_asof(df_right, on="int_id")

int_id,value,value_right
i64,str,i64
5,"""a""",1
10,"""b""",2


In [42]:
df_left.join_asof(
    df_right,
    on="int_id",
    coalesce=False,
)

int_id,value,int_id_right,value_right
i64,str,i64,i64
5,"""a""",4,1
10,"""b""",7,2


In [43]:
df_left.join_asof(
    df_right.rename({"int_id": "int_id_right"}),
    left_on="int_id",
    right_on="int_id_right",
)

int_id,value,int_id_right,value_right
i64,str,i64,i64
5,"""a""",4,1
10,"""b""",7,2


### Estrategias de unión inexacta en Polars: `df.join_asof()`

El método `df.join_asof()` permite unir DataFrames cuando los valores clave no coinciden exactamente. Polars ofrece tres estrategias principales para determinar cómo se selecciona la fila del DataFrame derecho:

- **backward** (por defecto):  
    Une con la última fila cuyo valor es igual o menor al valor de la fila izquierda.

- **forward**:  
    Une con la primera fila cuyo valor es igual o mayor al valor de la fila izquierda.

- **nearest**:  
    Une con la fila cuyo valor es el más cercano (menor diferencia absoluta) al valor de la fila izquierda.

Estas estrategias son especialmente útiles para series temporales o datos secuenciales donde la coincidencia exacta no es posible, pero se requiere la mejor aproximación.


In [44]:
print(df_left)
print(df_right)

shape: (2, 2)
┌────────┬───────┐
│ int_id ┆ value │
│ ---    ┆ ---   │
│ i64    ┆ str   │
╞════════╪═══════╡
│ 5      ┆ a     │
│ 10     ┆ b     │
└────────┴───────┘
shape: (3, 2)
┌────────┬───────┐
│ int_id ┆ value │
│ ---    ┆ ---   │
│ i64    ┆ i64   │
╞════════╪═══════╡
│ 4      ┆ 1     │
│ 7      ┆ 2     │
│ 12     ┆ 3     │
└────────┴───────┘


In [45]:
df_left.join_asof(
    df_right,
    on="int_id",
    tolerance=3,
    strategy="backward",
)

int_id,value,value_right
i64,str,i64
5,"""a""",1
10,"""b""",2


In [46]:
df_left.join_asof(
    df_right,
    on="int_id",
    tolerance=3,
    strategy="forward",
)

int_id,value,value_right
i64,str,i64
5,"""a""",2
10,"""b""",3


In [47]:
df_left.join_asof(
    df_right,
    on="int_id",
    tolerance=3,
    strategy="nearest",
)

int_id,value,value_right
i64,str,i64
5,"""a""",1
10,"""b""",3


## Additional Fine-Tuning
When you set the tolerance argument, rows are only joined if the nearest matching
value falls within a certain range. You can set the tolerance for numeric and temporal
data types. For temporal data types, use a datetime.timedelta or the duration
Strings (as we previously talked about in Table 13-5), such as "7d12h30m".

##  Use Case: Marketing Campaign Attribution

In [48]:
campaigns = pl.scan_csv(r"data\campaigns.csv")
campaigns.head(1).collect()

Campaign Name,Campaign Date,Product Type
str,str,str
"""Launch""","""2023-01-01 20:00:00""","""Electronics"""


In [49]:
campaigns.select(pl.col("Product Type").unique()).collect()

Product Type
str
"""Clothing"""
"""Furniture"""
"""Books"""
"""Electronics"""


In [50]:
transactions = pl.scan_csv(r"data\transactions.csv")
transactions.head(1).collect()

Sale Date,Product Type,Quantity
str,str,i64
"""2023-01-01 02:00:00.000000000""","""Books""",7


In [ ]:

transactions = transactions.with_columns(
    pl.col("Sale Date")
    .str.to_datetime("%Y-%m-%d %H:%M:%S%.f")
    .cast(pl.Datetime("us")),  # Diferencia: 'us' en vez de 'ms'
)

# La transformación de 'Campaign Date' es igual en ambos códigos.
campaigns = campaigns.with_columns(
    pl.col("Campaign Date").str.to_datetime("%Y-%m-%d %H:%M:%S"),
)

# El join_asof es igual en ambos códigos, solo cambia el nombre de la variable resultado.
sales_with_campaign_df = (
    transactions.sort("Sale Date")
    .join_asof(
        campaigns.sort("Campaign Date"),
        left_on="Sale Date",
        right_on="Campaign Date",
        by="Product Type",
        strategy="backward",
        tolerance="60d",
    )
    .collect()
)

sales_with_campaign_df

C:\Users\sergi\AppData\Local\Temp\ipykernel_15692\2609408275.py:25: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  .collect()


Sale Date,Product Type,Quantity,Campaign Name,Campaign Date
datetime[μs],str,i64,str,datetime[μs]
2023-01-01 01:26:12.558627,"""Electronics""",2,null,null
2023-01-01 02:00:00,"""Books""",7,null,null
2023-01-01 06:14:30.703535,"""Toys""",9,null,null
2023-01-01 06:52:25.117255,"""Clothing""",9,null,null
2023-01-01 07:44:50.234511,"""Books""",7,null,null
…,…,…,…,…
2023-12-31 15:45:29.296464,"""Clothing""",10,null,null
2023-12-31 18:15:09.765488,"""Toys""",4,null,null
2023-12-31 18:33:47.441372,"""Electronics""",7,null,null


In [54]:
(
    sales_with_campaign_df.group_by("Product Type", "Campaign Name")
    .agg(pl.col("Quantity").mean())
    .sort("Product Type", "Campaign Name")
)

Product Type,Campaign Name,Quantity
str,str,f64
"""Books""",null,5.527716
"""Clothing""",null,5.433385
"""Clothing""","""New Arrivals""",8.200581
"""Electronics""",null,5.486832
"""Electronics""","""Launch""",8.080775
"""Electronics""","""Seasonal Sale""",8.471406
"""Furniture""",null,5.430222
"""Furniture""","""Discount""",8.191888
"""Toys""",null,5.50318


In [55]:
campaigns.filter(pl.col("Product Type") == "Books").collect()

Campaign Name,Campaign Date,Product Type
str,datetime[μs],str
"""Clearance""",2023-12-31 21:00:00,"""Books"""


In [57]:
(
    transactions.filter(
        (pl.col("Product Type") == "Books")
        & (
            pl.col("Sale Date")
            > pl.lit("2023-12-31 21:00:00").str.to_datetime()
        )
    ).collect()
)

Sale Date,Product Type,Quantity
datetime[μs],str,i64


### Uniones No-Equi Inestables en Polars

Polars soporta actualmente las llamadas **non-equi joins**, que permiten unir DataFrames basándose en condiciones distintas a la igualdad, utilizando el método `df.join_where()`. Estas uniones están en fase experimental y no se cubren en este capítulo, pero es útil saber que existen para casos avanzados donde se requiere unir datos bajo reglas personalizadas más allá de la coincidencia exacta de valores.